In [3]:
import math
import numpy as np
import pandas as pd
validator = "Twinstake_processed.csv"
path = f"./{validator}"
df_data= pd.read_csv(path)

In [4]:
df_data['block_len'][1:].mean()

6.668996732026144

In [5]:
# df_data

In [6]:
total_missed = 0
indices = df_data['block_num']
for i in range(1,len(indices)):
    if int(indices[i]) != int(indices[i-1])+1:
        total_missed += 1
        print(i)

In [7]:
print(total_missed)
print(len(df_data))

0
307


In [8]:
df_data.reset_index(inplace=True)

In [9]:
print(df_data)

     index  block_num  Block_length_target  Goal_Bonded  \
0        0   12763285             7.233028       0.6666   
1        1   12763286             7.233028       0.6666   
2        2   12763287             7.233028       0.6666   
3        3   12763288             7.233028       0.6666   
4        4   12763289             7.233028       0.6666   
..     ...        ...                  ...          ...   
302    302   12763587             7.233028       0.6666   
303    303   12763588             7.233028       0.6666   
304    304   12763589             7.233028       0.6666   
305    305   12763590             7.233028       0.6666   
306    306   12763591             7.233028       0.6666   

     Inflation_Rate_Change  Min_Inflation_Rate  Max_Inflation_Rate  \
0                     0.13                0.07                 0.2   
1                     0.13                0.07                 0.2   
2                     0.13                0.07                 0.2   
3          

In [10]:
df_output = pd.DataFrame()
df_output['block Number'] = df_data.block_num
df_output['Atom Staked'] = df_data.atom_staked_v
df_output['Total_Rewards'] = ( df_data.commission_amt/(df_data.v_commission/100)) * 1E-6
df_output['Epoch Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Realized APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in APY'] = df_output.apply(lambda _: '', axis=1)
df_output['% Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)

print(df_output.head())

   block Number  Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0      12763285  185753.0002     624.804480                                  
1      12763286  185753.0002     624.812527                                  
2      12763287  185753.0002     624.820574                                  
3      12763288  185753.0002     624.828617                                  
4      12763289  185753.0002     624.836666                                  

  Realized APY Expected APY Absolute Difference in Rewards  \
0                                                            
1                                                            
2                                                            
3                                                            
4                                                            

  Absolute Difference in APY % Difference in Rewards  
0                                                     
1                                               

In [11]:
# Constants from Network


# Parameters to be varied by validator
for i in range(1,len(df_data)):

  Goal_Bonded                 = float(df_data.Goal_Bonded[i]) 
  Inflation_Rate_Change       = float(df_data.Inflation_Rate_Change[i]) 
  Min_Inflation_Rate          = float(df_data.Min_Inflation_Rate[i]) 
  Max_Inflation_Rate          = float(df_data.Max_Inflation_Rate[i]) 
  Block_length                = float(df_data.block_len[i]) 
  E_Block_length              = float(df_data.Block_length_target[i])
  Blocks_per_year             = float(df_data.Blocks_per_year[i]) 
  minProposerBonus            = float(df_data.min_proposer_bonus[i]) 
  maxProposerBonus            = float(df_data.max_proposer_bonus[i]) 
  Community_Tax               = float(df_data.community_tax[i]) 
    
  Min_Signatures              = float(df_data.Min_Signatures[i]) 
  Total_Fees_Per_Block        = float(df_data.total_block_fees[i]) * 1E-6
  Bonded_Ratio                = float(df_data.percent_staked[i]) /100
  stake                       = float(df_data.atom_staked_v[i]) 
  signatures_ratio            = float(df_data.sign_ratio[i]) 
  E_Inflation_Rate_BH         = float(df_data.inflation_rate[i]) 
  totalSupply                 = float(df_data.total_supply[i]) * 1E-6

  # E_current_Inflation_rate_change = (1 - Bonded_Ratio/Goal_Bonded) * Inflation_Rate_Change
  # E_Inflation_Rate_BH             = Prev_Inflation_Rate + E_current_Inflation_rate_change / Blocks_per_year

    # Check the new inflation rate is between min and max inflation rate
  if E_Inflation_Rate_BH > Max_Inflation_Rate:
    E_Inflation_Rate_BH = Max_Inflation_Rate
  elif E_Inflation_Rate_BH < Min_Inflation_Rate:
    E_Inflation_Rate_BH = Min_Inflation_Rate

  # Real_Inflation_Rate = E_Inflation_Rate_BH * Block_length/ E_Block_length

  E_validator         = stake / (totalSupply*Bonded_Ratio) * ((1+E_Inflation_Rate_BH)**(1/Blocks_per_year)-1) * totalSupply    # technically can cancel totalSupply but left for ease of understanding
  #E_validator         = stake / (totalSupply*Bonded_Ratio) * E_Inflation_Rate_BH/Blocks_per_year * totalSupply
  E_Proposer_Bonus                     = minProposerBonus + (signatures_ratio-Min_Signatures) / (1-Min_Signatures) * (maxProposerBonus-minProposerBonus)      
  E_Proposer_stake                     = totalSupply*Bonded_Ratio/175   # THIS IS POOR ESTIMATE AS VALIDATORS WITH MORE STAKE ARE SELECTED MORE FREQUENTLY, WOULD NEED ALL VALIDATOR'S DATA TO IMPROVE MODEL
  E_transaction_reward_per_atom_staked = Total_Fees_Per_Block * (1 - Community_Tax) / (totalSupply*Bonded_Ratio + E_Proposer_stake*E_Proposer_Bonus)

  E_ProposerTxFees                     = stake * (1 + E_Proposer_Bonus) * E_transaction_reward_per_atom_staked
  E_AttesterTxFees                     = stake * E_transaction_reward_per_atom_staked

  E_Attester = E_validator + E_AttesterTxFees
  E_Proposer = E_validator + E_ProposerTxFees

  P_Proposer = stake/totalSupply    # think this is true even with round robin selection
  P_attester = 1 - P_Proposer


  E_rewards = E_Proposer*P_Proposer + E_Attester*P_attester
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
  df_output['Expected Rewards'][i] = E_rewards
  df_output['Absolute Difference in Rewards'][i] = df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]
  df_output['% Difference in Rewards'][i] = (df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]) / df_output['Epoch Rewards'][i]*100

  APY                                            = E_rewards/stake*Blocks_per_year*E_Block_length/Block_length
  Realized_APY                                   = df_output['Epoch Rewards'][i]/stake*Blocks_per_year*E_Block_length/Block_length
  df_output['Expected APY'][i]                   = APY*100
  df_output['Realized APY'][i]                   = Realized_APY*100     
  df_output['Absolute Difference in Rewards'][i] = E_rewards - df_output['Epoch Rewards'][i]
  df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])

print(df_output)

/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_7006/160660359.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_7006/160660359.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Expected Rewards'][i] = E_rewards
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_7006/160660359.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

     block Number  Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0        12763285  185753.0002     624.804480                                  
1        12763286  185753.0002     624.812527      0.008047         0.008126   
2        12763287  185753.0002     624.820574      0.008047         0.008126   
3        12763288  185753.0002     624.828617      0.008043         0.008126   
4        12763289  185753.0002     624.836666      0.008049         0.008126   
..            ...          ...            ...           ...              ...   
302      12763587  185753.0002     627.710786      0.008048         0.008127   
303      12763588  185753.0002     627.718831      0.008045         0.008127   
304      12763589  185753.0002     627.726880      0.008049         0.008127   
305      12763590  185753.0002     627.734927      0.008048         0.008127   
306      12763591  185753.0002     627.742982      0.008054         0.008127   

    Realized APY Expected APY Absolute 

In [12]:
# df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/COSMOS/single-validators/Output Files/COSMOSValidationOutput-{validator}")

In [13]:
total_rewards = df_output['Epoch Rewards'][1:].sum()
expected_total = df_output['Expected Rewards'][1:].sum()

In [14]:
acual_total_rew_without_processing = (df_data.iloc[len(df_data)-1]['commission_amt']-df_data.iloc[0]['commission_amt'])/1e6/0.05
(acual_total_rew_without_processing-expected_total)/expected_total*100

89.06217431680507

In [15]:
expected_total

2.486802073139488

In [16]:
acual_total_rew_without_processing

4.701602070432902

In [17]:
(total_rewards-expected_total)/total_rewards*100

15.371755043980265

In [18]:
df_output[1:][df_output['Epoch Rewards'][1:]>0.010]

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
19,12763304,185753.0002,624.965489,0.016125,0.008126,42.789552,21.562469,-0.008,-21.227084,-49.6081
69,12763354,185753.0002,625.368037,0.016114,0.008126,40.776173,20.562985,-0.007988,-20.213188,-49.571076
124,12763409,185753.0002,625.810817,0.016109,0.008127,23.969614,12.092799,-0.007982,-11.876815,-49.549464
157,12763442,185753.0002,626.076562,0.016095,0.008127,42.343503,21.382162,-0.007967,-20.961341,-49.503087
179,12763464,185753.0002,626.253669,0.016107,0.008127,42.350466,21.368943,-0.00798,-20.981523,-49.542603
235,12763520,185753.0002,626.704637,0.016118,0.008127,40.099388,20.219769,-0.007991,-19.879619,-49.575867
265,12763550,185753.0002,626.946183,0.016112,0.008127,39.602546,19.976826,-0.007984,-19.62572,-49.556713
282,12763567,185753.0002,627.083030,0.016114,0.008127,42.792664,21.582993,-0.007987,-21.209671,-49.563802
290,12763575,185753.0002,627.147443,0.016101,0.008127,42.419128,21.412187,-0.007973,-21.006941,-49.522331
300,12763585,185753.0002,627.694689,0.482838,0.008127,1169.542507,19.686147,-0.474711,-1149.85636,-98.316765


In [19]:
df_output

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
0,12763285,185753.0002,624.804480,,,,,,,
1,12763286,185753.0002,624.812527,0.008047,0.008126,21.332553,21.542243,0.000079,0.20969,0.982958
2,12763287,185753.0002,624.820574,0.008047,0.008126,21.426337,21.636846,0.000079,0.210509,0.98248
3,12763288,185753.0002,624.828617,0.008043,0.008126,20.850605,21.065283,0.000083,0.214678,1.029602
4,12763289,185753.0002,624.836666,0.008049,0.008126,20.563555,20.760955,0.000077,0.1974,0.959953
...,...,...,...,...,...,...,...,...,...,...
302,12763587,185753.0002,627.710786,0.008048,0.008127,21.646185,21.859984,0.000079,0.213799,0.987698
303,12763588,185753.0002,627.718831,0.008045,0.008127,22.001944,22.226197,0.000082,0.224253,1.01924
304,12763589,185753.0002,627.726880,0.008049,0.008127,20.566634,20.767645,0.000079,0.201011,0.977366
305,12763590,185753.0002,627.734927,0.008048,0.008127,12.313466,12.435134,0.00008,0.121668,0.988088
